## Predicting Teacher Attrition Rates for Public Schools
**This notebook explores predicting teacher attrition rates for public schools in North Carolina.** 
* Public school racial compositions are also considered when making predictions in this notebook.
* The North Carolina Educational Attainment Data Repository for Machine Learning is located on Github at: https://github.com/wtubin/TeacherRetentionofNC

**For documentation on various Generalized Linear Models in Sklearn see:**
* http://scikit-learn.org/stable/modules/linear_model.html
* https://stackoverflow.com/questions/33845539/modelling-probabilities-in-a-regularized-logistic-regression-model-in-python

In [1]:
#!pip install xgboost
#!pip install pyshp

In [2]:
#import required Libraries
from __future__ import division
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV    
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn import metrics as mt
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import cross_validate
from scipy.stats import pearsonr
from sklearn.metrics import explained_variance_score
import xgboost
import shapefile as shp
%matplotlib inline


In [3]:
#The 'path' variable may also be a URL pointing to the proper raw file path on github: 

ML2017 = '../Machine Learning Datasets/PublicSchools2017_ML.csv'
ML2016 = '../../2016/Machine Learning Datasets/PublicSchools2016_ML.csv'
ML2015 =  '../../2015/Machine Learning Datasets/PublicSchools2015_ML.csv'
ML2014 =  '../../2014/Machine Learning Datasets/PublicSchools2014_ML.csv'

schData2017=pd.read_csv(ML2017, low_memory=False)
print('*********************************2017 ML Data*************************************')
schData2017.info()
schData2016=pd.read_csv(ML2016, low_memory=False)
print('*********************************2016 ML Data*************************************')
schData2016.info()

schData2015=pd.read_csv(ML2015, low_memory=False)
print('*********************************2015 ML Data*************************************')
schData2015.info()

schData2014=pd.read_csv(ML2014, low_memory=False)
print('*********************************2014 ML Data*************************************')
schData2014.info()

*********************************2017 ML Data*************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2468 entries, 0 to 2467
Columns: 396 entries, student_num to unit_code
dtypes: float64(352), int64(44)
memory usage: 7.5 MB
*********************************2016 ML Data*************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Columns: 376 entries, closed_ind to unit_code
dtypes: float64(298), int64(78)
memory usage: 7.0 MB
*********************************2015 ML Data*************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2432 entries, 0 to 2431
Columns: 405 entries, closed_ind to unit_code
dtypes: float64(344), int64(61)
memory usage: 7.5 MB
*********************************2014 ML Data*************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426 entries, 0 to 2425
Columns: 366 entries, closed_ind to unit_code
dtypes: float64(333), int64(

## Fix Year

In [4]:
schData2014["Year"]=2014
schData2015["Year"]=2015
schData2016["Year"]=2016
schData2017["Year"]=2017

In [5]:
# Combine the data sets
schData = pd.concat([schData2017,schData2016,schData2015,schData2014], axis=0, ignore_index=True, sort = True)

In [6]:
schData['Number of farm returns 75KLT100K'].describe()

count    9756.000000
mean       72.796228
std        63.113575
min         0.000000
25%        20.000000
50%        60.000000
75%       110.000000
max       240.000000
Name: Number of farm returns 75KLT100K, dtype: float64

In [7]:
#Missing Data Threshold (Per Column)
missingThreshold = 0.60

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25


#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = schData.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = schData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schData = schData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
schData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))


*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9756 entries, 0 to 9755
Columns: 412 entries, (TCE) prepared returns Ct 100KLT200K to wap_per_classroom
dtypes: float64(393), int64(19)
memory usage: 30.7 MB

Columns Deleted:  101


In [8]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values



*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
0,(TCE) prepared returns Ct 100KLT200K,4858
1,(TCE) prepared returns Ct 25KLT50K,2426
2,(TCE) prepared returns Ct 75KLT100K,2426
3,(VITA) prepared returns Ct 100KLT200K,2426
4,(VITA) prepared returns Ct 25KLT50K,2426
5,(VITA) prepared returns Ct 50KLT75K,2426
7,00_Size,2426
8,01_Size,4894
9,02_Size,2426
10,03_Size,2426


In [9]:
#Replace all remaining NaN with 0
schData = schData.fillna(0)

#Check for Missing values after final imputation 
missing_values = schData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values


### we are not remove the 0 1 year percentage for now.

In [10]:
zeroAttrition = schData[schData['_1yr_tchr_trnovr_pct']==0.0].index
schData = schData.drop(zeroAttrition,axis=0)
print('*********After: Removing rows with first year turn over percentage =0.00******')
schData.info(verbose=False)
print ('\r\nRows Deleted: ', len(zeroAttrition))

*********After: Removing rows with first year turn over percentage =0.00******
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9253 entries, 1 to 9755
Columns: 412 entries, (TCE) prepared returns Ct 100KLT200K to wap_per_classroom
dtypes: float64(393), int64(19)
memory usage: 29.2 MB

Rows Deleted:  503


## Visulization

### Heat map

In [11]:
#Credit: https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f
# https://catalog.data.gov/dataset/tiger-line-shapefile-2016-state-north-carolina-current-county-subdivision-state-based

# set seaborn style
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(10,6))
# import the North Carolina Map
shp_path = "./tl_2016_37_cousub/tl_2016_37_cousub.shp"
sf = shp.Reader(shp_path)


In [12]:
def plot_map_fill_multiples_ids_tone(sf, title, county,  
                                     print_id, color_ton, 
                                     bins, 
                                     x_lim = None, 
                                     y_lim = None, 
                                     figsize = (11,9)):
    '''
    Plot map with lim coordinates
    '''
        
    plt.figure(figsize = figsize)
    fig, ax = plt.subplots(figsize = figsize)
    fig.suptitle(title, fontsize=16)
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        ax.plot(x, y, 'k')
        for id in df.index:
            shape_ex = sf.shape(id)
            x_lon = np.zeros((len(shape_ex.points),1))
            y_lat = np.zeros((len(shape_ex.points),1))
            for ip in range(len(shape_ex.points)):
                x_lon[ip] = shape_ex.points[ip][0]
                y_lat[ip] = shape_ex.points[ip][1]
                ax.fill(x_lon,y_lat, color_ton[county1.index(id)])
            if print_id != False:
                x0 = np.mean(x_lon)
                y0 = np.mean(y_lat)
                plt.text(x0, y0, id, fontsize=10)
        if (x_lim != None) & (y_lim != None):     
                plt.xlim(x_lim)
                plt.ylim(y_lim)

In [13]:
# function to read shape file into panda data frame
def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df


In [14]:
def plot_countys_data(sf, title, countys, data=None, 
                      color=None, print_id=False):
    '''
    Plot map with selected comunes, using specific color
    '''
    
    color_ton, bins = calc_color(data, color)
    df = read_shapefile(sf)
    county_id = []
    for i in countys:
        #i = conv_comuna(i).upper()
        county_id.append(df[df.NAME ==i].index.get_values()[0])
    plot_map_fill_multiples_ids_tone(sf, title, county_id, 
                                     print_id, 
                                     color_ton, 
                                     bins, 
                                     x_lim = None, 
                                     y_lim = None, 
                                     figsize = (60,25));

In [15]:
mapdf = read_shapefile(sf)
mapdf.to_csv('North Carolina Map.csv',sep=',', index=False)

In [19]:
schData.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9253 entries, 1 to 9755
Data columns (total 412 columns):
(TCE) prepared returns Ct 100KLT200K                                               float64
(TCE) prepared returns Ct 25KLT50K                                                 float64
(TCE) prepared returns Ct 75KLT100K                                                float64
(VITA) prepared returns Ct 100KLT200K                                              float64
(VITA) prepared returns Ct 25KLT50K                                                float64
(VITA) prepared returns Ct 50KLT75K                                                float64
0-3 Years_LEA_Exp_Pct_Prin                                                         float64
00_Size                                                                            float64
01_Size                                                                            float64
02_Size                                                                  

In [17]:
title = '1 Year Teacher Attrition Distrubution on North Carolina'
data = census_17.PERSONAS
names = census_17.NOM_COMUNA
plot_countys_data(sf, title, names, data, 4, True)

NameError: name 'census_17' is not defined

In [ ]:
# function to calculate the colors on the map
def calc_color(data, color=None):
        if color   == 1: 
            color_sq =['#dadaebFF','#bcbddcF0','#9e9ac8F0','#807dbaF0','#6a51a3F0','#54278fF0']; 
            colors = 'Purples';
        elif color == 2: 
            color_sq = ['#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494']; 
            colors = 'YlGnBu';
        elif color == 3: 
            color_sq = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#636363','#252525']; 
            colors = 'Greys';
        elif color == 9: 
            color_sq = ['#ff0000','#ff0000','#ff0000','#ff0000','#ff0000','#ff0000']
        else:            
            color_sq = ['#ffffd4','#fee391','#fec44f','#fe9929','#d95f0e','#993404']; 
            colors = 'YlOrBr';
        new_data, bins = pd.qcut(data, 6, retbins=True,labels=list(range(6)))
        color_ton = []
        for val in new_data:
            color_ton.append(color_sq[val]) 
        if color != 9:
            colors = sns.color_palette(colors, n_colors=6)
            sns.palplot(colors, 0.6);
            for i in range(6):
                print ("\n"+str(i+1)+': '+str(int(bins[i]))+
                       " => "+str(int(bins[i+1])-1), end =" ")
            print("\n\n   1   2   3   4   5   6")    
        return color_ton, bins;

### Histogram

In [ ]:
plt.figure(figsize=(13,10))
_1yr_tchr_trnovr_pct = schData["_1yr_tchr_trnovr_pct"]
ax = sns.distplot(_1yr_tchr_trnovr_pct);
plt.title("Figure 1. North Carolina Public School First Year Teacher Turnover Percentage (2014-2017)")
plt.show()

In [ ]:
schData['lea_salary'] = schData['lea_total_expense_num']*schData['lea_salary_expense_pct']

plt.figure(figsize=(13,10))
lea_salary_expense_pct=schData.lea_salary_expense_pct
ax = sns.distplot(schData['lea_salary']);
plt.title("Figure 2. North Carolina Public School Lea Salary Expense (2014-2017)")
plt.show()

In [ ]:
# responseible variabl distribution
schData['_1yr_tchr_trnovr_pct'].describe()

## we may want to check the outliers

In [ ]:
sns.set_style("darkgrid")
data = schData
sns.lmplot(x="lea_salary", y="_1yr_tchr_trnovr_pct", hue = "Year",  data=data,height=8.27, aspect=11.7/8.27)
sns.despine()

#plt.title("Figure 3. North Carolina Public School One Year Teacher Attrition Rate vs. LEA Salary Expenditure (2014-2017)")
plt.ylabel('1st Year Teacher Attrition Rate')
plt.xlabel('Lea Salary')
#plt.show()
plt.savefig('Figure1.png')

In [ ]:
sns.lmplot(x="lea_total_expense_num", y="_1yr_tchr_trnovr_pct", hue="Year", data=schData,height=8.27, aspect=11.7/8.27)
sns.despine()

#plt.title("Figure 4. North Coralina Public School One Year Teacher Attrition Rate vs. LEA Total School Expenditure (2014-2017)")
plt.ylabel('1st Year Attrition Rate')
plt.xlabel('Lea Total Expense Num')
plt.savefig('Figure2.png')

In [ ]:
plt.figure(figsize=(14, 8))
ax = sns.boxplot(x="Year", y="_1yr_tchr_trnovr_pct", data=schData)
ax = sns.swarmplot(x="Year", y="_1yr_tchr_trnovr_pct", data=schData, color=".25")
plt.ylabel('1st Year Teacher Attrition Rate')
plt.xlabel('Year')
plt.savefig('Figure3.png')

In [ ]:
#schData.info(verbose=True)

## Feature Selection
* set max features we want to select

### Manually remove columns

* there columns is not what we are interested in, so we remove them manaually from data set

In [ ]:
# manually remove the columns

ManualDropFeatures = ['st_1yr_tchr_trnovr_pct','lea_1yr_tchr_trnovr_pct','PacificIslandPct','PacificIslandMalePct','PacificIslandFemalePct','HispanicPct'
                      ,'BlackPct','AsianPct','IndianPct','WhitePct','TwoOrMorePct','TwoOrMoreMalePct','TwoOrMoreFemalePct'

                      ,'lea_supplies_expense_pct'
                      ,'lea_services_expense_pct','lea_salary_expense_pct','lea_total_expense_num','lea_benefits_expense_pct'                    
                     ]

schData = schData.drop(ManualDropFeatures,axis=1)
print('*********After: Removing fetures we are not interested in ******')
schData.info(verbose=False)
print ('\r\nFeatures Deleted: ', len(ManualDropFeatures))

## These features have big number of the weight in feature selection, trying to remove them and see if it will be better

In [ ]:
schData = schData.drop(['Number of farm returns All','Number of farm returns 100KLT200K',
'Number of farm returns LT25K',
'Number of farm returns 25KLT50K',
'Number of farm returns 50KLT75K',
'Number of farm returns 75KLT100K',
'Number of farm returns GE200K'],axis=1)

In [ ]:
schData.info()

## Transformation

In [ ]:
# Get the X and y
y= schData['_1yr_tchr_trnovr_pct']
X = schData.drop(['_1yr_tchr_trnovr_pct'],axis=1)

In [ ]:
X.info()

In [ ]:
# Transform

saved_cols = X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)
#Save as data frames
X = pd.DataFrame(X)
X.columns = saved_cols


In [ ]:
# threthold for the feature selection
max_features =200
# backup the data set
schData_bak = schData
#schData = schData_bak

In [ ]:
# Feature Selection 
X_fs = X
y_fs = y


## Pearson Correlation

In [ ]:
# Pearson Correlation
feature_name = X_fs.columns.tolist()
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
        # replace NaN with 0
        cor_list = [0 if np.isnan(i) else i for i in cor_list]
        # feature name
        cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-max_features:]].columns.tolist()
        # feature selection? 0 for not select, 1 for select
        cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X_fs, y_fs)
print(str(len(cor_feature)), 'selected features')
print(pd.DataFrame( cor_feature))

## Linear Reggesion RFE

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=max_features, step=10, verbose=5)
rfe_selector.fit(X_fs, y_fs)
rfe_support = rfe_selector.get_support()
rfe_feature = X_fs.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(pd.DataFrame(rfe_feature))

## Linear Regression

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression

Linear = LinearRegression()

Linear_selector = SelectFromModel(Linear, threshold='1.25*median', max_features=max_features)
Linear_selector.fit(X_fs, y_fs)
Linear_support = Linear_selector.get_support()
Linear_feature = X_fs.loc[:,Linear_support].columns.tolist()
print(str(len(Linear_feature)), 'selected features')
print(pd.DataFrame(Linear_feature))

## Lasso

In [ ]:

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

from sklearn.linear_model import Ridge

LassoCV = LassoCV(cv = 5)

LassoCV_selector = SelectFromModel(LassoCV, threshold='1.25*median', max_features=max_features)
LassoCV_selector.fit(X_fs, y_fs)
LassoCV_support = LassoCV_selector.get_support()
LassoCV_feature = X_fs.loc[:,LassoCV_support].columns.tolist()
print(str(len(LassoCV_feature)), 'selected features')
print(pd.DataFrame(LassoCV_feature))

## Ridge Regression

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge

reg = Ridge()

reg_selector = SelectFromModel(reg, threshold='1.25*median', max_features=max_features)
reg_selector.fit(X_fs, y_fs)
reg_support = reg_selector.get_support()
reg_feature = X_fs.loc[:,reg_support].columns.tolist()
print(str(len(reg_feature)), 'selected features')
print(pd.DataFrame(reg_feature))

## Elastic Net Regression

In [ ]:
from sklearn.linear_model import ElasticNet

ELN = ElasticNet()

ELN_selector = SelectFromModel(ELN, threshold='1.25*median', max_features=max_features)
ELN_selector.fit(X_fs, y_fs)
ELN_support = ELN_selector.get_support()
ELN_feature = X_fs.loc[:,ELN_support].columns.tolist()
print(str(len(ELN_feature)), 'selected features')
print(pd.DataFrame(ELN_feature))


In [ ]:
## Summary
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name
                                     , 'Pearson':cor_support
                                     , 'Linear Regression':Linear_support
                                     , 'RFE':rfe_support
                                     , 'Lasso':LassoCV_support
                                     ,'Ridge':reg_support
                                     , 'Elastic Net':ELN_support
                                     
                                    })
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] ,
ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df

In [ ]:
# export feature selection to csv file
feature_selection_df.to_csv('Feature_Selection_Results.csv', sep=',', index=False)

In [ ]:
feature_to_remove = feature_selection_df[feature_selection_df['Total']<3]['Feature']
X = X.drop(feature_to_remove, axis=1)

#feature_to_remove
print ('\r\nFeatures Deleted: ', len(feature_to_remove))

## RFE Feature Importance

In [ ]:
# %%time

# df2_vif = pd.DataFrame()
# df2_vif["VIF Factor"] = [vif(X.values, i) for i in range(X.shape[1])]
# df2_vif["ABSVIFFactor"] = abs(df2_vif["VIF Factor"])
# df2_vif["features"] = X.columns
# df2_vif["indexes"] = range(0, len(X.columns))

In [ ]:
# df2_vif = df2_vif.sort_values('ABSVIFFactor', ascending=True)
# df2_vif.to_csv('VIF_Result_ML.csv', sep=',', index=False)

In [ ]:
# %%time



# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#     test_size=0.3,random_state =1 )

# #print("RFE CV Linear Regression 1st Pass")
# rfecvEstimator = LinearRegression()

# parameters = { 'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}

# #Create a grid search object  

# grid = GridSearchCV(estimator=rfecvEstimator
#                    #, n_jobs=8 # jobs to run in parallel
#                    #, verbose=0 # low verbosity
#                    , param_grid=parameters
#                   )

# #Perform hyperparameter search to find the best combination of parameters for our data using RFECV
# grid.fit(X_train, y_train)
# print("r2 / variance : ", grid.best_score_)
# print("Residual sum of squares: %.2f"
#               % np.mean((grid.predict(X_test) - y_test) ** 2))

In [ ]:

#Create Cross Validation Object with 10 folds with 80/20 train - test split
cv = ShuffleSplit(n_splits = 10, test_size=0.20, random_state=0)

In [ ]:
#Use mean absolute error (MAE) to score the regression models created 
#(the scale of MAE is identical to the response variable)
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, r2_score 
#from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, r2_scorer 

#Function for Root mean squared error
#https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

#Function for Mean Absolute Percentage Error (MAPE) - Untested
#Adapted from - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python
# revised from - https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
# still not working
def mape(y_actual, y_predicted): 
    mask = y_actual != 0           
    return (np.fabs(y_actual - y_predicted)/y_actual)[mask].mean() * 100
# Credit: https://alex.miller.im/posts/linear-model-custom-loss-function-regularization-python/
def mean_absolute_percentage_error(y_true, y_pred, sample_weights=None):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert len(y_true) == len(y_pred)
    
    if np.any(y_true==0):
        print("Found zeroes in y_true. MAPE undefined. Removing from set...")
        idx = np.where(y_true==0)
        y_true = np.delete(y_true, idx)
        y_pred = np.delete(y_pred, idx)
        if type(sample_weights) != type(None):
            sample_weights = np.array(sample_weights)
            sample_weights = np.delete(sample_weights, idx)
        
    if type(sample_weights) == type(None):
        return(np.mean(np.abs((y_true - y_pred) / y_true)))
    else:
        sample_weights = np.array(sample_weights)
        assert len(sample_weights) == len(y_true)
        return(100/sum(sample_weights)*np.dot(
                sample_weights, (np.abs((y_true - y_pred) / y_true))
        ))
    
#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(score_func=mean_squared_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)
#mape_scorer = make_scorer(score_func=mape, greater_is_better=False)
r2_scorer = make_scorer(score_func=r2_score, greater_is_better=True) 

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'MSE':  mse_scorer, 
                'RMSE': rmse_scorer,
                #'MAPE': mape_scorer,
                'R2': r2_scorer       
               } 

In [ ]:
from sklearn.model_selection import cross_validate

def EvaluateRegressionEstimator(regEstimator, X, y, cv):
    
    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_MSE'] = scores['test_MSE'] * -1
    #scores['test_MAPE'] = scores['test_MAPE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))

     #print mean MAE for all folds 
    mseAvg = scores['test_MSE'].mean()
    print_str = "The average MSE for all cv folds is: \t\t\t {mseAvg:.5}"
    print(print_str.format(mseAvg=mseAvg))
    
    #print mean test_MAPE for all folds
#     scores['test_MAPE'] = scores['test_MAPE']
#     mape_avg = scores['test_MAPE'].mean()
#     print_str = "The average MAE percentage (MAPE) for all cv folds is: \t {mape_avg:.5}"
#     print(print_str.format(mape_avg=mape_avg))

    #print mean MAE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))
    print('*********************************************************')
    
    #print mean R-squared for all folds 
    R2avg = scores['test_R2'].mean()
    print_str = "The average R-Squared for all cv folds is: \t\t {R2avg:.5}"
    print(print_str.format(R2avg=R2avg))
    print('*********************************************************')


    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['MSE'] = scores['test_MSE']
   # scoresResults['MAPE'] = scores['test_MAPE']
    scoresResults['RMSE'] = scores['test_RMSE']
    scoresResults['R2'] = scores['test_R2']
    return scoresResults


In [ ]:
%%time


print("RFE CV Linear Regression 1st Pass")
rfecvEstimator = LinearRegression()

parameters = { 'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]
             }

#Create a grid search object  
#from sklearn.model_selection import GridSearchCV
rfecvGridSearch = GridSearchCV(estimator=rfecvEstimator
                   , n_jobs=8 # jobs to run in parallel
                  , verbose=0 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   )

#Perform hyperparameter search to find the best combination of parameters for our data using RFECV
rfecvGridSearch.fit(X, y)

#Use the best parameters for our RFECV Linear Regression object
rfecvLinearEst = rfecvGridSearch.best_estimator_

#Recursive Feature Elimination
rfecv = RFECV(estimator=rfecvLinearEst, step=1, cv=cv,  verbose=1)
X_BestFeatures = rfecv.fit_transform(X, y)

#Print RFECV Details
print("Ranking", rfecv.ranking_)
print("Support", rfecv.support_)
print("Number of Features:", rfecv.n_features_)

print("Linear Regression Second Pass")
#create a pipeline to scale all of the data and perform logistic regression during each grid search step.
pipe = make_pipeline(StandardScaler(), LinearRegression())

#Define a range of hyper parameters for grid search
parameters = { 'linearregression__fit_intercept':[True,False], 'linearregression__normalize':[True,False]
              , 'linearregression__copy_X':[True, False]
              , 'linearregression__n_jobs':[None]
             }

#Perform the grid search using accuracy as a metric during cross validation.
grid = GridSearchCV(estimator=pipe, param_grid=parameters, cv=cv)

#Use the best features from recursive feature elimination during the grid search
grid.fit(X, y)


In [ ]:
#Print the parameterization of the best estimator
regEstimator = grid.best_estimator_

RFE_Results = EvaluateRegressionEstimator(grid.best_estimator_, X_BestFeatures, y, cv)

In [ ]:
#Load the model's coefficient weights and feature names into a dataframe sorted by weights
weights = regEstimator.named_steps['linearregression'].coef_.ravel()#[rfecv.get_support(indices=True)]
feature_names = X.columns#.values[rfecv.get_support(indices=True)]
#print(weights)

In [ ]:
#Load the model's coefficient weights and feature names into a dataframe sorted by weights
#weights = grid.best_estimator_.named_steps['linearregression'].coef_.ravel()
#feature_names = X.columns#.values[rfecv.get_support(indices=True)]

linreg_ft_imp_df = pd.DataFrame({'feature_names':feature_names, 'weights':weights,
                                 'absolute_weights': np.abs(weights)})
linreg_ft_imp_df.sort_values(by='absolute_weights', inplace=True, ascending=False )

#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

#linreg_ft_imp_df = linreg_ft_imp_df[linreg_ft_imp_df.feature_names != 'SchoolYear']

model_features = len(linreg_ft_imp_df.index)
print('Total Model Features: ' + str(model_features))

#Print all of the features selected by the model 
linreg_ft_imp_df


In [ ]:
#Get the top 75 coefficients
wt_plt_df = linreg_ft_imp_df.head(100)

model_features = len(wt_plt_df.index)
print('Total Model Features: ' + str(model_features))

# Examine categorical variables of interest  
import matplotlib.pyplot as plt
%matplotlib inline

#Plot the model's feature importances
# REFERENCE:  Eric Larson, https://github.com/eclarson/DataMiningNotebooks
plt.style.use('ggplot')

weights = pd.Series(wt_plt_df['weights'].values,index=wt_plt_df['feature_names'])
ax = weights.plot(kind='bar', figsize=(20,8))

ax.set_title("Top Feature Correlations")
ax.set_ylabel("Coefficient Magnitude\n(z-score)")
ax.set_xlabel("Feature Names")
plt.show()

In [ ]:
# export feature selection to csv file
linreg_ft_imp_df.to_csv('RFE_Feature_Importance_Results.csv', sep=',', index=False)

## Cross Validation
**Cross validation is performed using repeated holdout using ShuffleSplit()**
* Ten folds are used
* The split is: 90% training data and 10% test data
* A random seed is set so the same random test and training splits are used each time cross validation is performed.

In [ ]:
#Divide data into test and training splits
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.10, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state =1 )

## Custom Scorers for Evaluating Regression Models 

**All regression models created in this notebook are validated using the following metrics:**
* Mean Absolute Error (MAE)
* Root Mean Squared Error (RMSE) - https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
* Mean Absolute Percentage Error (MAPE) - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python

**For details on making scorers to return multiple mean error scores see:**
* http://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html
* https://github.com/scikit-learn/scikit-learn/pull/7388
* https://github.com/drorata/multiscorer

In [ ]:
# it may not make sense to use MAPE in our project based on below articles:
# http://www.catchbull.com/catchblogs/why-mape-doesnt-work


## Regression Model Evaluation
** All regression models are evaluated using the regression model evaluation function below: ** 
* The following regression evaluation function uses the cross validation object and the custom scorers in the two cells above in combination with sklearn.model_selection's cross_validate function to perform cross validation for regression estimators.
* The cross validation object above uses a random seed to ensure that all regression estimators are tested on the same randomly selected records for each cross validation fold.
* Custom scorers are created using the three chosen mean error scores and passed into cross_validate(), so all three scores are calcualted using a single call to cross_validate().
* All of this functionality is wrapped within the custom EvaluateRegressionEstimator() function below so multiple regression models may be tested using the same test / train cv data and evaluation scores producing a consistent output for each model without the need to re-write the same code over and over. 

## Baseline Regression Model

**Linear Regression is used to create a baseline model.  Since linear regression may predict response variable values outside the range of the training data's response variable, we create a linear regression estimator with graduation rate predictions clipped 0% and 100%. For details see:**
* http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator 
* https://github.com/scikit-learn/scikit-learn/issues/6950
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
* https://github.com/scikit-learn-contrib/project-template/blob/master/skltemplate/template.py
* https://stackoverflow.com/questions/44234682/how-to-use-sklearn-when-target-variable-is-a-proportion

In [ ]:
#Make new estimator compatible for use with GridSearchCV() and cross_validate()
# -  Cap predict function for LinearRegression between 0 and 100
# -  See: Roll your own estimator links above for details. 
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression

class CappedLinearRegression(LinearRegression):

    def predict(self, X):
        return np.clip(super(CappedLinearRegression, self).predict(X), 0, 100) 

### Linear Regression Grid Search
** Here we perform a grid search testing 40 models to find the best parameters for our Linear Regression model based on Mean Absolute Error.  See more on parameter tuning with grid search here:**
* http://scikit-learn.org/stable/modules/grid_search.html

In [ ]:
#Create a Linear Regression object and perform a grid search to find the best parameters
linreg = CappedLinearRegression()
parameters = {'normalize':(True,False), 'fit_intercept':(True,False),'normalize': (True, False),}

#Create a grid search object using the  
regGridSearch = GridSearchCV(estimator=linreg
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

In [ ]:
#Print the parameterization of the best estimator
regEstimator = regGridSearch.best_estimator_
#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
CappedLinear_Results = EvaluateRegressionEstimator(regEstimator, X, y, cv)

In [ ]:
regEstimator = regGridSearch.best_estimator_

regEstimator.fit(X_train, y_train)
yhat = regEstimator.predict(X_test)
print("Yhat Max: ", yhat.max())

## Support Vector Regression
**This model uses Support Vector Machines for regression of continuous variables (SVR). Please see documentation here:"**
* http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
* http://scikit-learn.org/stable/modules/svm.html

In [ ]:
#Create a Linear regression object and perform a grid search to find the best parameters
from sklearn.svm import SVR
reg = SVR()

#Set up SVR parameters to test (WARNING: Creates 320 models!!!) 
costs = [0.001, 0.1]
defGamma = 1 / X.shape[1]  #This is the default value for the gamma parameter
gammas = [defGamma, 0.1]
kernels = ['rbf','linear']
parameters = {'C': costs, 'gamma' : gammas, 'kernel': kernels}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=reg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

In [ ]:
#Display the best estimator parameters
regGridSearch.best_estimator_

In [ ]:
# from sklearn.svm import SVR

#Create a regression estimator with best parameters for cross validation
# regEstimator = SVR(C=0.001, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
#                    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
SVR_Results=EvaluateRegressionEstimator(regEstimator, X, y, cv)

In [ ]:
#Do we predict graduation rates greater than 100%?
# regEstimator = SVR(C=0.001, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
#                    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

regEstimator.fit(X_train, y_train)
yhat = regEstimator.predict(X_test)
print("Yhat Max: ", yhat.max())

## Graduation Rate - Lasso Regression
**This model uses Lasso regression (L1 Norm). Please see documentation here:"**
* **Caution!** - See documentation for fit_intercept, normalize, and copy_X. Lasso can over-write your X data!
* Lasso may also perform scaling as well.  Please see docs!
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Lasso

reg = Lasso(fit_intercept=True, normalize=True,copy_X=True
          , max_iter=10000, precompute=True, tol=0.0001, random_state=0)

#Test parameters 
alpha = [0.001, 0.1, 1, 10, 20]
selection = ['cyclic','random']
warm_start = [True, False]
parameters = {'alpha': alpha, 'selection': selection, 'warm_start': warm_start}

#Create a grid search object using the parameters above
regGridSearch = GridSearchCV(estimator=reg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

In [ ]:
#Display the best estimator parameters
regGridSearch.best_estimator_

In [ ]:
#Create a regression estimator with best parameters for cross validation
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
Lasso_Results = EvaluateRegressionEstimator(regEstimator, X, y, cv)

In [ ]:
#Do we predict graduation rates greater than 100%?
# regEstimator = Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=10000,
#    normalize=True, positive=False, precompute=True, random_state=0,
#    selection='cyclic', tol=0.0001, warm_start=True)
regEstimator = regGridSearch.best_estimator_

regEstimator.fit(X_train, y_train)
yhat = regEstimator.predict(X_test)
print("Yhat Max: ", yhat.max())

## Ridge Regression
**This model uses Ridge regression (L2 Norm). Please see documentation here:"**
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [ ]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Ridge

reg = Ridge(fit_intercept=True, normalize=True,copy_X=True
          , max_iter=10000, tol=0.0001, random_state=0)

#Test parameters 
alpha = [0.001, 0.1, 1, 5, 10, 20]
solver = [ 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
parameters = {'alpha': alpha, 'solver': solver}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=reg
                   , n_jobs=10 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X, y)

In [ ]:
#Display the best estimator parameters
regGridSearch.best_estimator_

In [ ]:
%%time
#Create a regression estimator with best parameters for cross validation
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
Ridge_Results = EvaluateRegressionEstimator(regEstimator, X, y, cv)

In [ ]:
#Do we predict graduation rates greater than 100%?
# regEstimator = Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=10000,
#    normalize=True, random_state=0, solver='saga', tol=0.0001)
regEstimator = regGridSearch.best_estimator_

regEstimator.fit(X_train, y_train)
yhat = regEstimator.predict(X_test)
print("Yhat Max: ", yhat.max())

## Elastic Net Regression
**This model uses Elastic Net Regression (L1 and L2 Norm mixing). Please see documentation here:"**
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [ ]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import ElasticNet

elasticNet = ElasticNet(fit_intercept=True, normalize=True, precompute=True, copy_X=True
          , max_iter=10000, tol=0.0001, random_state=0)
 
#Test parameters
l1_ratio = [0.001, 0.01, 0.1, 0.5, 0.75, 1]
alpha = [0.001, 0.1, 1, 10]
selection = ['cyclic','random']
warm_start = [True, False]
parameters = {'l1_ratio': l1_ratio, 'alpha': alpha, 'selection': selection, 'warm_start': warm_start}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
elasticNetGridSearch = GridSearchCV(estimator=elasticNet
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
elasticNetGridSearch.fit(X, y)

In [ ]:
#Display the best estimator parameters
elasticNetGridSearch.best_estimator_

In [ ]:
#Create a regression estimator with best parameters for cross validation
elasticNetEstimator = elasticNetGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
elastic_Results = EvaluateRegressionEstimator(elasticNetEstimator, X, y, cv)

In [ ]:
#Do we predict graduation rates greater than 100%?
elasticNetEstimator =  elasticNetGridSearch.best_estimator_

elasticNetEstimator.fit(X_train, y_train)
yhat = elasticNetEstimator.predict(X_test)
print("Yhat Max: ", yhat.max())

## XGBoost

In [ ]:
# credit: https://www.kaggle.com/mburakergenc/predictions-with-xgboost-and-linear-regression


In [ ]:
regr = LinearRegression()

In [ ]:
 #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state =1 )

In [ ]:
# Let's try XGboost algorithm to see if we can get better results
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [ ]:
traindf, testdf = train_test_split(X_train, test_size = 0.3)
xgb.fit(X_train,y_train)

In [ ]:
predictions = xgb.predict(X_test)
print(explained_variance_score(predictions,y_test))